In [2]:
import pandas as pd
import os, cv2, keras
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
from keras import layers

In [3]:
df = pd.read_csv('cropped/kanji_labels.csv')

In [4]:
df

,image_id,labels
0,image_0,三
1,image_1,世
2,image_2,人
3,image_3,子
4,image_4,者
...,...,...
41144,image_41144,中
41145,image_41145,今
41146,image_41146,気
41147,image_41147,居


In [5]:
X_=[]
y_=[]
for image_id, labels in df.values:
    image_fn = 'cropped/kanji_cropped/{}.jpg'.format(image_id)
    image = Image.open(image_fn)
    image = np.asarray(image)
    X_.append(image)
    y_.append(labels)

In [6]:
X_ = np.array(X_)
y_ = np.array(y_)

In [7]:
X_.shape

(41149, 100, 100)

In [8]:
y_.shape

(41149,)

In [9]:
unique, counts = np.unique(y_, return_counts=True)

In [10]:
len(counts)

103

In [11]:
i = 0
classes = []
for c in counts:
    if(c >= 10 and unique[i]):
        cl = unique[i]
        classes.append(cl)
    i+=1

In [12]:
j = 0
yy1_ = []
XX1_ = []
for y in y_:
    if(y in classes):
        yy1_.append(y)
        XX1_.append(X_[j])
    j+= 1

In [13]:
X_ = np.array(XX1_)
y_ = np.array(yy1_)

In [14]:
print(X_.shape)
print(y_.shape)

(41149, 100, 100)
(41149,)


In [15]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn import metrics

from keras.models import Sequential, Model,load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, BatchNormalization, MaxPool2D, MaxPooling2D, AveragePooling2D, Input
from keras.optimizers import Adam
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint
from datetime import datetime

from keras import backend as K

In [16]:
leinst = LabelEncoder()
yyinst = leinst.fit_transform(y_)

# x_train, x_test, y_train, y_test = train_test_split(X_, yyinst, test_size=0.2, shuffle=True, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(X_, yyinst, test_size=0.2, shuffle=True, stratify=yyinst)

# x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, random_state=42)
x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size=0.5, shuffle=True, stratify=y_test)

print(x_train.shape)
print(x_test.shape)
print(x_validation.shape)


num_classes = len(leinst.classes_)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_validation = to_categorical(y_validation, num_classes=num_classes)

(32919, 100, 100)
(4115, 100, 100)
(4115, 100, 100)


In [17]:
num_classes

103

In [18]:
K.clear_session()
def Kuzushiji_Classifier(in_):
    model_ = Conv2D(64,kernel_size=(3, 3),activation='relu')(in_)
    model_ = BatchNormalization()(model_)
    model_ = Conv2D(32,kernel_size=(3, 3),activation='relu')(model_)
    model_ = MaxPooling2D(pool_size=(2, 2))(model_)
    model_ = BatchNormalization()(model_)
    model_ = Dropout(0.4)(model_)
    
    model_ = Conv2D(32,kernel_size=(3, 3), strides=2,activation='relu')(model_)
    model_ = MaxPooling2D(pool_size=(2, 2))(model_)
    model_ = BatchNormalization()(model_)
    model_ = Dropout(0.4)(model_)
    
    model_ = Flatten()(model_)
    model_ = Dense(128, activation='relu')(model_)
    model_ = Dropout(0.4)(model_)
    model_ = Dense(num_classes, activation='softmax')(model_)
    return model_

In [19]:
Input_Sample = Input(shape=(100, 100, 1))
Output_ = Kuzushiji_Classifier(Input_Sample)
Model_Enhancer = Model(inputs=Input_Sample, outputs=Output_)

In [20]:
Model_Enhancer.compile(loss = "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
Model_Enhancer.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 98, 98, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        18464     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128   

In [21]:
checkpointer = ModelCheckpoint('model_kanji1.h5', verbose=0,mode='auto', monitor='val_accuracy',save_best_only=True)

In [22]:
ModelHistory = Model_Enhancer.fit(x_train, y_train,
          batch_size=16,
          epochs=20,
          verbose=1,
          callbacks=[checkpointer],
          validation_data=(x_validation, y_validation))

Epoch 1/20
2058/2058 [==============================] - 606s 294ms/step - loss: 1.8304 - accuracy: 0.5740 - val_loss: 0.5627 - val_accuracy: 0.8603
Epoch 2/20
2058/2058 [==============================] - 623s 303ms/step - loss: 0.8201 - accuracy: 0.7846 - val_loss: 0.3561 - val_accuracy: 0.9108
Epoch 3/20
2058/2058 [==============================] - 610s 297ms/step - loss: 0.6281 - accuracy: 0.8289 - val_loss: 0.3065 - val_accuracy: 0.9217
Epoch 4/20
2058/2058 [==============================] - 702s 341ms/step - loss: 0.5318 - accuracy: 0.8508 - val_loss: 0.2770 - val_accuracy: 0.9337
Epoch 5/20
2058/2058 [==============================] - 713s 347ms/step - loss: 0.4665 - accuracy: 0.8671 - val_loss: 0.2598 - val_accuracy: 0.9339
Epoch 6/20
2058/2058 [==============================] - 655s 318ms/step - loss: 0.4228 - accuracy: 0.8788 - val_loss: 0.2549 - val_accuracy: 0.9390
Epoch 7/20
2058/2058 [==============================] - 725s 352ms/step - loss: 0.3797 - accuracy: 0.8898 - val_

In [23]:
train_score = Model_Enhancer.evaluate(x_train, y_train, verbose=1)
print("Training Accuracy: ", train_score[1])

1029/1029 [==============================] - 105s 102ms/step - loss: 0.0234 - accuracy: 0.9954
Training Accuracy:  0.9953522086143494


In [24]:
val_score = Model_Enhancer.evaluate(x_validation, y_validation, verbose=1)
print("Validation Accuracy: ", val_score[1])

129/129 [==============================] - 13s 101ms/step - loss: 0.1894 - accuracy: 0.9536
Validation Accuracy:  0.9535844326019287


In [25]:
test_score = Model_Enhancer.evaluate(x_test, y_test, verbose=1)
print("Test Accuracy: ", test_score[1])

129/129 [==============================] - 13s 101ms/step - loss: 0.2562 - accuracy: 0.9402
Test Accuracy:  0.9402186870574951


In [26]:
y_pred = Model_Enhancer.predict(x_test)

129/129 [==============================] - 14s 105ms/step


In [27]:
i = np.argmax(y_pred[0])
print("Prediction: ", leinst.classes_[i])

Prediction:  右


In [28]:
i = np.argmax(y_test[0])
print("Prediction: ", leinst.classes_[i])

Prediction:  右


In [29]:
from sklearn.metrics import f1_score

In [30]:
y_true = np.argmax(y_test, axis=1)

In [31]:
y_pr = np.argmax(y_pred, axis=1)

In [32]:
f1 = f1_score(y_true, y_pr, average=None)

In [33]:
len(f1)

103

In [34]:
from sklearn.metrics import classification_report

In [35]:
n = 0
labels = []
for clas in leinst.classes_:
    labels.append(n)
    # print(clas, n)
    n += 1

In [36]:
report = classification_report(y_true, y_pr, labels=labels, target_names=leinst.classes_)
report_dict = classification_report(y_true, y_pr, labels=labels, target_names=leinst.classes_, output_dict=True)

In [37]:
print(report)

              precision    recall  f1-score   support

           一       0.99      0.99      0.99       133
           七       1.00      0.76      0.86        25
           三       0.94      0.97      0.95        64
           上       0.96      0.92      0.94        59
           下       0.98      0.98      0.98        42
           世       0.97      0.95      0.96        41
           中       0.92      0.94      0.93        47
           之       0.79      1.00      0.88        22
           也       0.91      0.89      0.90        36
           事       0.95      0.97      0.96       106
           二       0.96      1.00      0.98        52
           五       0.91      0.93      0.92        44
           人       0.96      0.96      0.96       136
           今       0.98      0.98      0.98        47
           仕       0.95      1.00      0.98        21
           付       1.00      0.90      0.95        20
           何       0.96      0.98      0.97        46
           候       0.94    